In [2]:
import os
import sys
import warnings
import pandas as pd
import numpy as np

warnings.filterwarnings("ignore", category=UserWarning, module="pandas")

PROJECT_ROOT = r"D:\demand_forecasting_system"
SRC_DIR = os.path.join(PROJECT_ROOT, "src")
if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)
 
from tasks.load_csv import load_all_csv
load_csv_files = load_all_csv()  # if

2025-10-21 21:06:55,275 | INFO | data_pipeline | Loaded CSV 'blinkit_inventory.csv' with 75172 rows, 4 columns


In [3]:
df_inventory = load_csv_files['blinkit_inventory']

In [3]:
print(df_inventory.info())
print(df_inventory.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75172 entries, 0 to 75171
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   product_id      75172 non-null  int64 
 1   date            75172 non-null  object
 2   stock_received  75172 non-null  int64 
 3   damaged_stock   75172 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 2.3+ MB
None
   product_id        date  stock_received  damaged_stock
0      153019  17-03-2023               4              2
1      848226  17-03-2023               4              2
2      965755  17-03-2023               1              0
3       39154  17-03-2023               4              0
4       34186  17-03-2023               3              2


Drop duplicates & unnecessary spaces

In [4]:
df_inventory.columns = df_inventory.columns.str.strip()
df_inventory.drop_duplicates(inplace=True)


Handle missing values

In [5]:
# Fill missing numeric columns with 0
df_inventory['stock_received'] = df_inventory['stock_received'].fillna(0)
df_inventory['damaged_stock'] = df_inventory['damaged_stock'].fillna(0)

# Drop rows where product_id or date is missing (they're essential)
df_inventory.dropna(subset=['product_id', 'date'], inplace=True)


Convert datatypes

In [6]:
df_inventory['product_id'] = df_inventory['product_id'].astype(int)
df_inventory['date'] = pd.to_datetime(df_inventory['date'], format='%d-%m-%Y', errors='coerce')



In [7]:
for col in ['stock_received', 'damaged_stock']:
    upper_limit = df_inventory[col].quantile(0.99)
    df_inventory.loc[df_inventory[col] > upper_limit, col] = upper_limit


In [8]:
print(df_inventory.isnull().sum())
print(df_inventory.dtypes)
print(df_inventory.shape)


product_id        0
date              0
stock_received    0
damaged_stock     0
dtype: int64
product_id                 int64
date              datetime64[ns]
stock_received             int64
damaged_stock              int64
dtype: object
(75172, 4)


In [9]:
import os

# Define target path
save_path = r"D:\demand_forecasting_system\data\processed\blinkit_inventory_clean.csv"

# Create directory if it doesn't exist
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# Save the cleaned DataFrame
df_inventory.to_csv(save_path, index=False)

print(f"Cleaned data saved successfully at: {save_path}")

Cleaned data saved successfully at: D:\demand_forecasting_system\data\processed\blinkit_inventory_clean.csv
